In [135]:
import random
from bs4 import BeautifulSoup
import re
import requests
import pandas as pd
import math

header_name = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/77.0.3865.90 Safari/537.36'
acronym_to_name_dict = {'ATL': 'Atlanta',
                        'WSH': 'Washington',
                        'NYM': 'New York Mets',
                        'PHI': 'Philadelphia',
                        'MIA': 'Miami',
                        'STL': 'St. Louis',
                        'MIL': 'Milwaukee',
                        'CHC': 'Chicago Cubs',
                        'CIN': 'Cincinatti',
                        'PIT': 'Pittsburg',
                        'LAD': 'Los Angeles Dodgers',
                        'ARI': 'Arizona',
                        'SF': 'San Francisco',
                        'COL': 'Colorado',
                        'SD': 'San Diego',
                        'NYY': 'New York Yankees',
                        'TB': 'Tampa Bay',
                        'BOS': 'Boston',
                        'TOR': 'Toronto',
                        'BAL': 'Baltimore',
                        'MIN': 'Minnesota',
                        'CLE': 'Cleveland',
                        'CHW': 'Chicago White Sox',
                        'KC': 'Kansas City',
                        'DET': 'Detroit',
                        'HOU': 'Houston',
                        'OAK': 'Oakland',
                        'TEX': 'Texas',
                        'LAA': 'Los Angeles Angels',
                        'SEA': 'Seattle'}
acronym_to_name_dict = {'HOU': 'Houston'}
mlb_teams = ['braves', 'nationals', 'mets', 'phillies', 'marlins', 
         'cardinals', 'brewers', 'cubs', 'reds', 'pirates', 
         'dodgers', 'd-backs', 'giants', 'rockies', 'padres', 
         'yankees', 'rays', 'red sox', 'blue jays', 'orioles', 'twins',
         'indians', 'white sox', 'royals', 'tigers', 'astros', 'athletics',
         'rangers', 'angels', 'mariners']

In [113]:
class Batter:
    """Class that holds information for a Batter.
    
    Attributes:
        team: team name of player
        name: player name
        singles: fraction of plate appearances ending in a single
        doubles: fraction of plate appearances ending in a double
        triples: fraction of plate appearances ending in a triple
        home_runs: fraction of plate appearances ending in a home run
        walks: fraction of plate appearances ending in a walk
    """
    def __init__(self, team, name, singles, doubles, triples, home_runs, walks):
        """Initializes values for this class"""
        
        self.team = team
        self.name = name
        self.singles = singles
        self.doubles = singles + doubles
        self.triples = singles + doubles + triples
        self.home_runs = singles + doubles + triples + home_runs
        self.walks = singles + doubles + triples + home_runs + walks
        self.cum_singles = 0
        self.cum_doubles = 0
        self.cum_triples = 0
        self.cum_home_runs = 0
        self.cum_walks = 0
        self.cum_pa = 0
        self.cum_runs = 0
        self.cum_rbi = 0
                
    def avg(self):
        """Calculates avg of a player
        
        Returns:
            float: batting average in decimal form
        """
        if(self.cum_pa-self.cum_walks>0):
            return(float((self.cum_singles+self.cum_doubles+self.cum_triples+self.cum_home_runs)/(self.cum_pa-self.cum_walks)))
        else:
            return 0    
    def obp(self):
        """Calculates on base percentage of a player
        
        Returns:
            float: opb in decimal form"""
        if(self.cum_pa>0):
            return((self.cum_singles+self.cum_doubles+self.cum_triples+self.cum_home_runs+self.cum_walks)/(self.cum_pa))
        else:
            return 0
    def slg(self):
        """Calculates slugging percentage of a player
        
        Returns:
            float: slg in decimal form"""
        if(self.cum_pa-self.cum_walks>0):
            return((self.cum_singles+2*self.cum_doubles+3*self.cum_triples+4*self.cum_home_runs)/(self.cum_pa-self.cum_walks))
        else:
            return 0
    def ops(self):
        """Calculates on base plus slugging of a player
        
        Returns:
            float: ops in decimal form"""
        
        return(self.slg() + self.obp())
    def print_stats(self):
        """Prints some stats of a player"""

        print("Avg: " + str(round(self.avg(), 3)))
        print("OBP: " + str(round(self.obp(), 3)))
        print("SLG: " + str(round(self.slg(), 3)))
        print("OPS: " + str(round(self.ops(), 3)))
        print("HR: " + str(self.cum_home_runs))
        
class Pitcher(Batter):
    """Class that holds information for a pitcher.
    
    Attributes:
        team: team name of player
        name: player name
        position: player position
        singles: fraction of plate appearances ending in a single
        doubles: fraction of plate appearances ending in a double
        triples: fraction of plate appearances ending in a triple
        home_runs: fraction of plate appearances ending in a home run
        walks: fraction of plate appearances ending in a walk
    """
    def __init__(self, team, name, singles, doubles, triples, home_runs, walks):
        """Initializes values for this class"""
        
        Batter.__init__(self, team, name, singles, doubles, triples, home_runs, walks)
        
class Team:
    """Class that holds information for a team.
    
    Attributes:
        name: team name
        p1-p9: instances of player class
        roster: batting lineup of team in list form
        index: position in batting order of current batter
        runners: list representing whether a runner is on each base
        runs: number of runs fr a team in a game
        wins: number of wins for a team
        losses: number of losses for a team
        cumulative runs: total runs for a team over a span of games
    """
    def __init__(self, name, batters, pitchers):
        """Initializes values for this class"""
        self.name = name
        self.batters = batters
        self.pitchers = pitchers
        self.pitching_staff = pitchers
        self.index = 0
        self.runners = [None, None, None, 0]    # first base-hom plate. 0 for base empty, 1 for runner on
        self.runs = 0
        self.wins = 0
        self.losses = 0
        self.cum_runs = 0
        self.cum_runs_allowed = 0
        
        def make_lineup(batters):
            order = {}
            for batter in batters:
                order.update({batter: batter.walks})
            order = [k for k, v in sorted(order.items(), key=lambda item: item[1], reverse=True)]
            return(order[0:9])
        
        def choose_pitcher(pitchers):
            order = {}
            for pitcher in pitchers:
                order.update({pitcher: pitcher.walks})
            order = [k for k, v in sorted(order.items(), key=lambda item: item[1])]
            return(order[0])
        
        self.lineup = make_lineup(batters)
        self.pitcher = choose_pitcher(pitchers)
        
    def restart(self):
        """Resets values for position in lineup, runners on base, and runs if a game is to be started"""
        self.index = 0
        self.runners = [None, None, None, 0]
        self.runs = 0
    def reset_runners(self):
        """Resets baserunners after each half inning"""
        self.runners = [None, None, None, 0]
        
class Game:
    """Class that holds information for a Game.
    
    Attributes:
        away_team: away team name
        home_team: home team name
    """
    
    def __init__(self, away_team, home_team):
        """Initializes values for this class"""
        self.away_team = away_team
        self.home_team = home_team
        
    def get_result(self, batter, pitcher, num):
        """Using player attributes and a random number, a result is created for a plate appearance
        
        Args:
            batter: instance of batter class
            pitcher: instance of pitcher class
            num: random number between 0 and 2
        Returns:
            str: refers to result based on random number (ie "walk")
        """
        batter.cum_pa+=1
        pitcher.cum_pa+=1
        
        if(num<batter.singles):
            batter.cum_singles+=1
            pitcher.cum_singles+=1
            return "single"
        elif(num<batter.doubles):
            batter.cum_doubles+=1
            pitcher.cum_doubles+=1
            return "double"
        elif(num<batter.triples):
            batter.cum_triples+=1
            pitcher.cum_triples+=1
            return "triple"
        elif(num<batter.home_runs):
            batter.cum_home_runs+=1
            pitcher.cum_home_runs+=1
            return "home_run"
        elif(num<batter.walks):
            batter.cum_walks+=1
            pitcher.cum_walks+=1
            return "walk"
        elif(num<1):
            return "out"
        elif(num<pitcher.singles+1):
            batter.cum_singles+=1
            pitcher.cum_singles+=1
            return "single"
        elif(num<pitcher.doubles+1):
            batter.cum_doubles+=1
            pitcher.cum_doubles+=1
            return "double"
        elif(num<pitcher.triples+1):
            batter.cum_triples+=1
            pitcher.cum_triples+=1
            return "triple"
        elif(num<pitcher.home_runs+1):
            batter.cum_home_runs+=1
            pitcher.cum_home_runs+=1
            return "home_run"
        elif(num<pitcher.walks+1):
            batter.cum_walks+=1
            pitcher.cum_walks+=1
            return "walk"
        else:
            return "out"
        
    def move_runners(self, team, pitcher, result):
        """Based on the result of a batter's plate appearance, runners are moved and runs may be added to team's total 
        
        Args:
            team: instance team that the batter was on
            pitcher: pitcher pitching to team
            result: str of result for batter (ie "walk")
        """
        if(result=="single"):
            if(team.runners[2]!=None):
                team.runners[2].cum_runs+=1
                pitcher.cum_runs+=1
                team.lineup[team.index].cum_rbi+=1
                team.runners[3]+=1
                
            team.runners[2] = team.runners[1]
            team.runners[1] = team.runners[0]
            team.runners[0] = team.lineup[team.index]

        elif(result=="double"):
            if(team.runners[2]!=None):
                team.runners[2].cum_runs+=1
                pitcher.cum_runs+=1
                team.lineup[team.index].cum_rbi+=1
                team.runners[3]+=1
            if(team.runners[1]!=None):
                team.runners[1].cum_runs+=1
                pitcher.cum_runs+=1
                team.lineup[team.index].cum_rbi+=1
                team.runners[3]+=1
                
            team.runners[2] = team.runners[0]
            team.runners[1] = team.lineup[team.index]
            team.runners[0] = None

        elif(result=="triple"):
            if(team.runners[2]!=None):
                team.runners[2].cum_runs+=1
                pitcher.cum_runs+=1
                team.lineup[team.index].cum_rbi+=1
                team.runners[3]+=1
            if(team.runners[1]!=None):
                team.runners[1].cum_runs+=1
                pitcher.cum_runs+=1
                team.lineup[team.index].cum_rbi+=1
                team.runners[3]+=1
            if(team.runners[0]!=None):
                team.runners[0].cum_runs+=1
                pitcher.cum_runs+=1
                team.lineup[team.index].cum_rbi+=1
                team.runners[3]+=1
                
            team.runners[2] = team.lineup[team.index]
            team.runners[1] = None
            team.runners[0] = None

        elif(result=="home_run"):
            team.lineup[team.index].cum_runs+=1
            pitcher.cum_runs+=1
            team.lineup[team.index].cum_rbi+=1
            team.runners[3]+=1
            
            if(team.runners[2]!=None):
                team.runners[2].cum_runs+=1
                pitcher.cum_runs+=1
                team.lineup[team.index].cum_rbi+=1
                team.runners[3]+=1
            if(team.runners[1]!=None):
                team.runners[1].cum_runs+=1
                pitcher.cum_runs+=1
                team.lineup[team.index].cum_rbi+=1
                team.runners[3]+=1
            if(team.runners[0]!=None):
                team.runners[0].cum_runs+=1
                pitcher.cum_runs+=1
                team.lineup[team.index].cum_rbi+=1
                team.runners[3]+=1
                
            team.runners[2] = None
            team.runners[1] = None
            team.runners[0] = None

        elif(result=="walk"):
            temp_2 = team.runners[2]
            temp_1 = team.runners[1]
            temp_0 = team.runners[0]
            
            team.runners[0] = team.lineup[team.index]
            
            if(temp_0!=None):
                team.runners[1] = temp_0
                
                if(temp_1!=None):
                    team.runners[2] = temp_1
                    
                    if(temp_2!=None):
                        temp_2.cum_runs+=1
                        pitcher.cum_runs+=1
                        team.lineup[team.index].cum_rbi+=1
                        team.runners[3]+=1

        if(team.runners[3]>0):
            team.runs+=team.runners[3]
            team.runners[3] = 0  
            
    def print_result(self):
        """Printed score of a game""" 
        print("final score: " + self.away_team.name + " " + str(self.away_team.runs) + " " + self.home_team.name + " " + str(self.home_team.runs))     
    
    def print_record(self):
        """Printed record of teams""" 
        print(self.away_team.name + ": " + str(self.away_team.wins) + "-" + str(self.away_team.losses))
        print(self.home_team.name + ": " + str(self.home_team.wins) + "-" + str(self.home_team.losses))     
   
    def play(self):
        """A game between two instances of the team class is played""" 
        self.away_team.restart()
        self.home_team.restart() 
        inning = 1
        
        while(inning<10 or self.away_team.runs==self.home_team.runs):
            for side in [self.away_team, self.home_team]:
                outs = 0
                
                if(side==self.away_team):
                    pitcher = self.home_team.pitcher
                elif(side==self.home_team):
                    pitcher = self.away_team.pitcher

                while(outs<3):
                    num = random.uniform(0,2)
                    result = self.get_result(side.lineup[side.index], pitcher, num)
                    if(result=="out"):
                        outs+=1
                    else:
                        self.move_runners(side, pitcher, result)
                        
                    side.index+=1
                    side.index = side.index%8
                    
                side.reset_runners()
            inning+=1
        
        if(self.away_team.runs>self.home_team.runs):
            self.away_team.wins+=1
            self.home_team.losses+=1
        elif(self.away_team.runs<self.home_team.runs):
            self.away_team.losses+=1
            self.home_team.wins+=1
        
        self.away_team.cum_runs+=self.away_team.runs
        self.away_team.cum_runs_allowed+=self.home_team.runs
        self.home_team.cum_runs+=self.home_team.runs
        self.home_team.cum_runs_allowed+=self.away_team.runs             
        
class Season(Game):
    """Class that holds information for a Season of games.
    
    Attributes:
        away_team: away team name
        home_team: home team name
    """
    
    def __init__(self, away_team, home_team):
        """Initializes values for this class"""
        self.away_team = away_team
        self.home_team = home_team
        
    def simulate_season(self, games):
        """A season of games is simulated
        
        Args:
            games: number of games to be played
        """
        for game in range(1, games+1):
            g = Game(self.away_team, self.home_team)
            g.play() 
            
    def reset_season(self, teams):
        
        for team in teams:
            team.__init__(team.name, team.batters, team.pitchers)
                        

In [4]:
def convert_int(row, column):
    
    if(column in row.index):
         if(row[column]!=''):
            return(int(row[column]))
         else:
            return(0)
        
def convert_float(row, column):
    
    if(column in row.index):
        if(row[column]!=''):
            return(float(row[column]))
        else:
            return(0.0)

In [108]:
url = 'http://www.espn.com/mlb/player/splits/_/id/31283/type/batting3'

headers = {'User-Agent': header_name}

platoon_dict = {}
data_columns = ['Split', 'Year', 'Tm', 'team_ID', 'Pos', 'POS', 'pos_season', 'Name', 'player', 'Age', 'age', 'G', 'PA', 'AB', 
                'R', 'H', '2B', '3B', 'HR', 'SB', 'CS', 'BB', 'SO', 'HBP']



source = requests.get(url, headers=headers)
soup = BeautifulSoup(source.content, 'html.parser')
left_columns = ['Name', 'Bats']
right_columns = []
name = soup.h1.get_text()

for bio in soup.findAll('div', attrs={'class': 'player-bio'}):
    for info in bio.findAll('ul', attrs={'class': 'general-info'}):
        bats = info.get_text().split('Bats')[1][2]

for table in soup.findAll('table', attrs={'class': 'tablehead'}):
    labels = table.find('tr', attrs={'class': 'colhead'})
    
    for header_value in labels.findAll('td'):
        left_columns.append(header_value.get_text()+'L')
        right_columns.append(header_value.get_text()+'R')
                
    df = pd.DataFrame(columns=(left_columns+right_columns))            
    player_values = [name, bats]
    
    for column_value in table.findAll('tr', attrs={'class': ['oddrow', 'evenrow']}):
        if(column_value.td.get_text() in ['vs. Left', 'vs. Right']):
            for value in column_value.findAll('td'):
                player_values.append(value.get_text())
     
    df = df.append(pd.Series(player_values, index=df.columns), ignore_index=True)

df.drop(['OverallL', 'OverallR'], axis=1, inplace=True)   
print(df)

               Name Bats  ABL  RL   HL 2BL 3BL HRL RBIL BBL  ... RBIR  BBR  \
0  Christian Yelich    L  503  82  148  30   3  25   75  53  ...  213  175   

  HBPR  SOR SBR CSR  AVGR  OBPR  SLGR  OPSR  
0   12  282  51   5  .318  .410  .582  .992  

[1 rows x 34 columns]


In [5]:
url = 'http://www.espn.com/mlb/history/leaders/_/breakdown/season/year/2019/start/'
links = [url+str(i) for i in range(1, 350, 50)]
headers = {'User-Agent': header_name}

batter_stats = pd.DataFrame()

for (num, link) in enumerate(links):
    source = requests.get(link, headers=headers)
    soup = BeautifulSoup(source.content, 'html.parser')
    table = soup.find('table', attrs={'class': 'tablehead'})
    if(num==0):
        column_headers = []
        table_headers = table.find('tr', attrs={'class': 'colhead'})
        for header_value in table_headers.findAll('td'):
            column_headers.append(header_value.get_text())

        column_headers[0] = 'ID'
        batter_stats = pd.DataFrame(columns=column_headers)
        
    df = pd.DataFrame(columns=column_headers)            


    for player in table.find_all('tr', attrs={'class': re.compile('row player-10-')}):
        player_values = []
        player_values.append(player['class'][1].split('-')[2])

        values =  player.findAll('td')
        values.pop(0)
        for value in values:
            player_values.append(value.get_text())

        df = df.append(pd.Series(player_values, index=df.columns), ignore_index=True)
    
    batter_stats = pd.concat([batter_stats, df], ignore_index=True)

print(batter_stats)

        ID            PLAYER YRS    G   AB    R    H  2B 3B  HR RBI  BB   SO  \
0     6524    Howie Kendrick  14  121  334   61  115  23  1  17  62  27   49   
1    33184      Tim Anderson   4  123  498   81  167  32  0  18  56  15  109   
2    39572       Luis Arraez   1   92  326   54  109  20  1   4  28  36   29   
3    29703    Donovan Solano   6   81  215   27   71  13  1   4  23  10   49   
4    31283  Christian Yelich   7  130  489  100  161  29  3  44  97  80  118   
..     ...               ...  ..  ...  ...  ...  ...  .. ..  ..  ..  ..  ...   
341  32819     Lewis Brinson   3   75  226   15   39   9  1   0  15  13   74   
342  30755      Keon Broxton   5  100  204   24   34   4  0   6  16  20  104   
343  32657       Mike Zunino   7   90  266   30   44  10  1   9  32  20   98   
344   5921       Jeff Mathis  15   86  228   17   36   9  0   2  12  15   87   
345  32890       Travis Shaw   5   86  230   22   36   5  0   7  16  36   89   

     SB CS    BA  
0     2  1  .344  
1

In [6]:
url = 'http://www.espn.com/mlb/history/leaders/_/type/pitching/breakdown/season/year/2019/sort/wins/start/'
links = [url+str(i) for i in range(1, 500, 50)]
headers = {'User-Agent': header_name}

pitcher_stats = pd.DataFrame()

for (num, link) in enumerate(links):
    source = requests.get(link, headers=headers)
    soup = BeautifulSoup(source.content, 'html.parser')
    table = soup.find('table', attrs={'class': 'tablehead'})
    if(num==0):
        column_headers = []
        table_headers = table.find('tr', attrs={'class': 'colhead'})
        for header_value in table_headers.findAll('td'):
            column_headers.append(header_value.get_text())

        column_headers[0] = 'ID'
        pitcher_stats = pd.DataFrame(columns=column_headers)
        
    df = pd.DataFrame(columns=column_headers)            


    for player in table.find_all('tr', attrs={'class': re.compile('row player-10-')}):
        player_values = []
        player_values.append(player['class'][1].split('-')[2])

        values =  player.findAll('td')
        values.pop(0)
        for value in values:
            player_values.append(value.get_text())

        df = df.append(pd.Series(player_values, index=df.columns), ignore_index=True)
    
    pitcher_stats = pd.concat([pitcher_stats, df], ignore_index=True)

print(pitcher_stats)

        ID             PLAYER YRS   G  GS CG SH     IP    H  ER  BB   SO   W  \
0     6341   Justin Verlander  15  34  34  2  1  223.0  137  64  42  300  21   
1    32081        Gerrit Cole   7  33  33  0  0  212.1  142  59  48  326  20   
2    32675  Eduardo Rodriguez   5  34  34  0  0  203.1  195  86  75  213  19   
3     5883       Zack Greinke  16  33  33  0  0  208.2  175  68  30  187  18   
4    30373  Stephen Strasburg  10  33  33  0  0  209.0  161  77  56  251  18   
..     ...                ...  ..  ..  .. .. ..    ...  ...  ..  ..  ...  ..   
495  35009        Paul Sewald   3  17   0  0  0   19.2   18  10   3   22   1   
496  35098     Ryan Carpenter   2   9   9  0  0   40.2   61  42  13   25   1   
497  35135       Tanner Scott   3  28   0  0  0   26.1   28  14  19   37   1   
498  35328       Pedro Payano   1   6   4  0  0   22.0   26  14  15   17   1   
499  35391       Luke Farrell   3   9   1  0  0   13.1    6   4   3   12   1   

     L SV   ERA  
0    6  0  2.58  
1  

In [7]:
string_columns = ['Pos', 'Name', 'PLAYER']
int_columns = ['ID', 'Age', 'G', 'PA', 'AB', 'R', 'H', '2B', '3B', 'HR',
               'RBI', 'SB', 'CS', 'BB', 'SO', 'OPS+', 'TB', 'GDP', 'HBP', 'SH', 'SF', 'IBB']
float_columns = ['BA', 'OBP', 'SLG', 'OPS']

for column in batter_stats.columns:
    if(column in int_columns):
        batter_stats[column] = batter_stats.apply(lambda row: convert_int(row, column), axis=1)
    elif(column in float_columns):
        batter_stats[column] = batter_stats.apply(lambda row: convert_float(row, column), axis=1)

In [8]:
string_columns = ['Pos', 'Name', 'PLAYER']
int_columns = ['ID', 'Age', 'W', 'L', 'G', 'GS', 'GF', 'CG', 'SHO', 'SV', 'H', 'R',
               'ER', 'HR', 'BB', 'IBB', 'SO', 'HBP', 'BK', 'WP', 'BF', 'ERA+']
float_columns = ['W-L%', 'ERA', 'IP', 'FIP', 'WHIP', 'H9', 'HR9', 'BB9', 'SO9',
                'SO/W']


for column in pitcher_stats.columns:
    if(column in int_columns):
        pitcher_stats[column] = pitcher_stats.apply(lambda row: convert_int(row, column), axis=1)
    elif(column in float_columns):
        pitcher_stats[column] = pitcher_stats.apply(lambda row: convert_float(row, column), axis=1)

In [122]:
url_dict = {}
url_start = 'https://www.espn.com/mlb/team/roster/_/name/'
team_dict = {}
for acronym, city in acronym_to_name_dict.items():
    url_dict.update({url_start+acronym.lower(): city})

for url, city in url_dict.items():
    headers = {'User-Agent': header_name}
    source = requests.get(url, headers=headers)
    soup = BeautifulSoup(source.content, 'html.parser')
    player_id_dict = {}
    pitchers = []
    batters = []

    for player in soup.findAll('td', attrs={'class': 'Table__TD'}):
        row = player.find('a', attrs={'class': 'AnchorLink'})
        if(row and row.get_text()!=''):
            player_id_dict.update({row.get_text(): int(row['href'].split('/')[-1])})

    for name, ID in player_id_dict.items():
        if(ID in pitcher_stats['ID'].tolist()):
            stats = pitcher_stats.loc[pitcher_stats['ID']== ID]
            print(stats)
            ip = math.modf(stats['IP'])
            outs = int(ip[0]*10 + ip[1]*3)
            total_batters = outs + stats['BB'].values[0] + stats['H'].values[0]
            hit_ratio = stats['H'].values[0]/total_batters
            singles = round(hit_ratio*.64, 3)
            doubles = round(hit_ratio*.2, 3)
            triples = round(hit_ratio*.02, 3)
            home_runs = round(hit_ratio*.14, 3)
            walks = round(stats['BB'].values[0]/total_batters, 3)

            pitchers.append(Pitcher(city, name, singles, doubles, triples, home_runs, walks))

        elif(ID in batter_stats['ID'].tolist()):
            stats = batter_stats.loc[batter_stats['ID']== ID]
            num_singles = stats['H'].values[0]-stats['2B'].values[0]-stats['3B'].values[0]-stats['HR'].values[0]
            pa = stats['AB'].values[0] + stats['BB'].values[0]
            singles = round(num_singles/pa, 3)
            doubles = round(stats['2B'].values[0]/pa, 3)
            triples = round(stats['3B'].values[0]/pa, 3)
            home_runs = round(stats['HR'].values[0]/pa, 3)
            walks = round(stats['BB'].values[0]/pa, 3)

            batters.append(Batter(city, name, singles, doubles, triples, home_runs, walks))
  
    team_dict.update({city: Team(city, batters, pitchers)})

        ID       PLAYER YRS   G  GS  CG SH    IP   H  ER  BB  SO  W  L  SV  \
301  34882  Joe Biagini   4  63   0   0  0  64.2  71  33  26  60  3  2   1   

      ERA  
301  4.59  


NameError: name 'pos' is not defined

In [94]:
Astros =Team('Astros', astros_batters, astros_pitchers)
Astros2 =Team('Astros', astros_batters, astros_pitchers)
for n in Astros.lineup:
    print(n.name)
print(Astros.pitcher.name)

IndexError: list index out of range

In [63]:
g = Game(team_dict['Atlanta'], team_dict['Kansas City'])
g.play()
g.print_result()

final score: Astros 3 Astros 2


In [120]:
s = Season(team_dict['Atlanta'], team_dict['Kansas City'])

s.simulate_season(162)
s.print_record()
s.reset_season([team_dict['Atlanta'], team_dict['Kansas City']])

Atlanta: 87-75
Kansas City: 75-87


In [172]:
url = 'https://www.mlb.com/news/projecting-every-mlb-lineup-rotation'
headers = {'User-Agent': header_name}
source = requests.get(url, headers=headers)
soup = BeautifulSoup(source.content, 'html.parser')

starters = {}

for body in soup.findAll('div', attrs={'class': 'article-item__body'}):
    pitchers = []
    batters = []
    
    for team in body.findAll('p'):
        if(team.strong and team.strong.get_text().lower() in mlb_teams):
            current_team = team.strong.get_text().lower()
            for batter in team.findAll(['forge-entity', 'a']):
                batters.append(batter.get_text())
        elif(team.strong and team.strong.get_text()=='Pitchers'):
            for pitcher in team.findAll(['forge-entity', 'a']):
                pitchers.append(pitcher.get_text())
        elif(team.strong and team.strong.get_text()=='Closer:'):
            closer = team.find(['forge-entity', 'a']).get_text()
        else:
            continue

        starters.update({current_team: {'lineup': batters, 'rotation': pitchers, 'closer': closer}})

In [173]:
print(starters)

{'blue jays': {'lineup': ['Bo Bichette', 'Cavan Biggio', 'Lourdes Gurriel Jr.', 'Teoscar Hernández', 'Vladimir Guerrero Jr.', 'Randal Grichuk', 'Travis Shaw', 'Rowdy Tellez', 'Danny Jansen', 'Austin Hays', 'Anthony Santander', 'Chris Davis', 'Renato Núñez', 'Rio Ruiz', 'Pedro Severino', 'Hanser Alberto', 'Dwight Smith Jr', 'José Iglesias', 'Austin Meadows', 'Yandy Díaz', 'Ji-Man Choi', 'Hunter Renfroe', 'Yoshitomo Tsutsugo', 'Brandon Lowe', 'Mike Zunino', 'Kevin Kiermaier', 'Willy Adames', 'Andrew Benintendi', 'Xander Bogaerts', 'Rafael Devers', 'J.D. Martinez', 'Mitch Moreland', 'Christian Vázquez', 'Jackie Bradley Jr.', 'Kevin Pillar', 'José Peraza', 'DJ LeMahieu', 'Brett Gardner', 'Gleyber Torres', 'Gary Sánchez', 'Luke Voit', 'Miguel Andújar', 'Mike Tauchman', 'Gio Urshela', 'Clint Frazier', 'Francisco Lindor', 'Oscar Mercado', 'Carlos Santana', 'Franmil Reyes', 'José Ramírez', 'Jordan Luplow', 'César Hernández', 'Roberto Pérez', 'Delino DeShields', 'Whit Merrifield', 'Adalberto Mo